In [1]:
import csv
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sns

In [2]:
class LocallyWeightedLR():
    
    def __init__(self, X, y, tau):
        self.X = X
        self.y = y
        self.tau = tau 
        
    def compute_weights(self, x):
        m = self.X.shape[0]
        W = np.eye(m)
        x = np.hstack((1, x))
        
        for i in range(m):
            diff = self.X[i] - x
            W[i, i] = np.exp(-diff.T.dot(diff) / (2 * self.tau ** 2))
            
        return W
    
    def compute_theta(self, x):
        W = self.compute_weights(x)
        x = np.hstack((1, x))
        XTWX = self.X.T.dot(W).dot(self.X)
        theta = np.linalg.inv(XTWX).dot(self.X.T).dot(W).dot(self.y)
        return theta
    
    # prediction for an input x
    # also return the local linear regression parameters (theta) for this x.
    def predict(self, x):
        theta = self.compute_theta(x)
        x = np.hstack((1, x))
        prediction = x.dot(theta)
        return prediction, theta

In [ ]:
co2_df = pd.read_csv('../datasets/co2-long-term-concentration.csv')
lpi_df = pd.read_csv('../datasets/global-living-planet-index.csv')